# Usage example

In [1]:
# Run before execution for Colab
import os

try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

try:
    import pypulceq

    HAS_PYPULCEQ = True
except:
    HAS_PYPULCEQ = False

# set environment variable
if IN_COLAB and not (HAS_PYPULCEQ):
    print("Installing PyPulCeq in COLAB")
    !pip install git+https://github.com/INFN-PREDATOR/pypulceq.git@main
else:
    print("Skipping installation")

/home/local/IMAGO7/mcencini/.conda/envs/deep-mr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Skipping installation


Pypulceq can be used to convert ``pypulseq.Sequence`` objects to the GE interpreter (``TOPPE``) file format.

The package can be imported and used in Python scripts. For example, we can convert a Cartesian 3D SPGR Pulseq sequence as follows:

1. Design the sequence using [PyPulseq](https://github.com/imr-framework/pypulseq)

In [2]:
# imports
import pypulceq

# design a sequence object - here we use a pre-packaged example
seq = pypulceq.demo.design_gre(fov=(32, 32), mtx=(32, 32))  # 1x1x1.2 mm3 voxel size

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 93.04it/s]


Timing check passed successfully


2. Perform actual conversion

In [3]:
pypulceq.seq2ge("cart_toppe", seq, verbose=True, nviews=32, nslices=32)

Removing duplicates from seq object...	done!

Selecting all blocks...	done!

Getting scan dynamics...	done!

Getting parent blocks...	done! Found 7 parent blocks.

Setting parent blocks amplitudes to max...	done!

Segment labels not found; attempt to determine it automatically...	done! Found 2 segments.

Converting Pulseq blocks to TOPPE modules...	done!

Writing sequence modules...	done!

Writing modules list...	done!

Writing group list...	done!

Writing scan loop...	

/home/local/IMAGO7/mcencini/M/PREDATOR/pypulceq/src/pypulceq/_interp.py:110: UserWarning: Gradient area for parent block 1 were not preserved after interpolating to GE raster time (in: 125.417 1/m, out: 125.400). Did you wrap all gradient events in trap4ge?
  warnings.warn(f"{warning_msg}")


done!

Writing entry file...	done!

preflightcheck: Checking max 10s SAR, 
time interval 0-10s (scan duration: 13s)
time interval 5-15s (scan duration: 13s)


	Predicted peak 10s SAR in a 150 lbs subject: 0.0 W/kg
Archive and clean-up...	done!

Sequence file cart_toppe ready for execution on GE scanners



This function can be used inside PyPulseq design routines to export the same sequence for either Siemens or GE intepreters, e.g.:

In [4]:
def mygredesign(fname, fov, mtx, interpreter):
    """
    Vendor agnostic 3D GRE.

    Parameters
    ----------
    fname : str
        Sequence filename ('fname.seq' for Siemens, 'fname.tar' for GEHC').
    fov : tuple
        Image field of view (in-plane, slab).
    mtx : tuple
        Image matrix size (nx=ny, nz).
    interpreter : str
        Desired interpreter ("Siemens" or "GEHC")

    """
    # design code block (replace with actual PyPulseq functions)
    seq = pypulceq.demo.design_gre(fov=fov, mtx=mtx)

    # 'interpreter' agnostic export section
    if interpreter == "Siemens":
        seq.write(f"{fname}.seq")
    elif interpreter == "GEHC":
        pypulceq.seq2ge(fname, seq, verbose=True, nviews=mtx[0], nslices=mtx[1])

In alternative, we provide a Command Line Interface to convert existing ``.seq`` files:

In [5]:
seq.write("cart_pypulseq.seq")  # assume we saved the sequence on disk

'ca0e7cb1bfc2c6f680c3612d0e41449f'

In [8]:
!seq2ge --output-name "cart_toppe_cli" --input-file "cart_pypulseq.seq" --nviews 32 --nslices 32 --verbose True

Reading .seq file...	done!

Selecting all blocks...	done!

Getting scan dynamics...	done!

Getting parent blocks...	done! Found 7 parent blocks.

Setting parent blocks amplitudes to max...	done!

Segment labels not found; attempt to determine it automatically...	done! Found 2 segments.

/home/local/IMAGO7/mcencini/M/PREDATOR/pypulceq/src/pypulceq/_interp.py:110: UserWarning: Gradient area for parent block 1 were not preserved after interpolating to GE raster time (in: 125.417 1/m, out: 125.400). Did you wrap all gradient events in trap4ge?
  warnings.warn(f"{warning_msg}")
Converting Pulseq blocks to TOPPE modules...	done!

Writing sequence modules...	done!

Writing modules list...	done!

Writing group list...	done!

Writing scan loop...	done!

Writing entry file...	done!

preflightcheck: Checking max 10s SAR, 
time interval 0-10s (scan duration: 13s)
time interval 5-15s (scan duration: 13s)


	Predicted peak 10s SAR in a 150 lbs subject: 0.0 W/kg
Archive and clean-up...	done!

Sequenc

A more detailed description of ``seq2ceq`` CLI can be showed by typing:

In [9]:
!seq2ge --help

Usage: seq2ge [OPTIONS]

  Convert a Pulseq file (http://pulseq.github.io/) to a set of files that can
  be executed on GE scanners using the TOPPE interpreter (v6).

Options:
  --output-name TEXT         Name of output .tar sequence file  [required]
  --input-file TEXT          Path on disk to input Pulseq file (.seq) to be
                             converted  [required]
  --sequence-path TEXT       location of sequence files on scanner  [default:
                             /usr/g/research/pulseq/v6/seq2ge/]
  --nviews INTEGER           number of views, i.e., frequency encodings for a
                             single k-space volume (e.g., phase encoding
                             lines).  [default: 600]
  --nslices INTEGER          number of slices / slab-encodings.  [default:
                             2048]
  --verbose BOOLEAN          Display information.  [default: True]
  --ignore-trigger BOOLEAN   ignore TTL pulses in sequence  [default: False]
  --ignore-segments BO